In [1]:
from DP.utils import fisher_gradient, fisher_information_privatized, binom_derivative
import cvxpy as cp
import numpy as np
from scipy.stats import binom

In [2]:
n = 4
theta = 0.5
epsilon = 1

In [4]:
def objective_func(Q):
    p_theta = binom.pmf(np.arange(n + 1), n, theta)
    p_theta_dot = [binom_derivative(i, n, theta) for i in range(n + 1)]

    numerator = cp.power(Q @ p_theta_dot, 2)
    return cp.sum(numerator)

Q_var = cp.Variable(shape=(n+1, n+1))
objective = cp.Maximize(objective_func(Q_var))

constraints = []
constraints += [Q_var >= 0]

for j in range(n + 1):
    constraints += [cp.sum(Q_var[:, j]) == 1]

exp_eps = np.exp(epsilon)
for i in range(n+1):
    for j in range(n+1):
        if i < n+1 - 1:
            constraints += [Q_var[i, j] - exp_eps * Q_var[i + 1, j] <= 0]
        if i > 0:
            constraints += [Q_var[i, j] - exp_eps * Q_var[i - 1, j] <= 0]

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to maximize a function that is convex.